# Preprocessing des images .TIF du dataset lnria Aerial Image Labeling Dataset

5000x5000 px en résolution 0.3 m /px vers 1024x1024 px en résolution 0.6 m /px

seulement les images sélectionnées à l'oeil dans le fichier prepro.txt selon les critères suivants :
- toit plat 
- toit gris sombre ou noir 

iteration 24/08
- faire la conversion en resolution 0.6m/x
- faire l'annotation
- découper les images avec gdalwarp, également les annotations json
-> renverser l'ordre découpage annotations pour gagner du temps et annoter directement les grandes images 

In [1]:
import os 
from osgeo import gdal 
import json
import copy
import base64
from PIL import Image

In [2]:
dataset_dir = r"C:\Users\VArri\Documents\Rooftop\dataset\dataset\dataset"

train_dir = os.path.join(dataset_dir, 'train', 'images')
test_dir  = os.path.join(dataset_dir, 'test', 'images')

## Convert GeoTIFF to 0.6 m resolution GeoTIFF

In [ ]:
def res(dataset_dir, res_dir, file_name, res=0.6):
    file_path = os.path.join(dataset_dir, file_name)
    name, ext = file_name.split('.')
    res_path = os.path.join(res_dir, name+'_s.'+ext)
    !gdalwarp -tr $res $res $file_path $res_path

In [ ]:
tmp_path = os.path.join(dataset_dir, 'tmp')
if not os.path.isdir(tmp_path):
    os.mkdir(tmp_path)

lst = os.listdir(train_dir)
for i in lst:
    if i.startswith('austin') or i.startswith('chicago') or i.startswith('vienna'):
        res(train_dir, tmp_path, i, res=0.6)
        
test_lst = os.listdir(test_dir)
for j in test_lst:
    if j.startswith('bellingham') or j.startswith('bloomington') or j.startswith('sfo'):
        res(test_dir, tmp_path, j, res=0.6)

## Convert GeoTIFF to 1024 px side GeoTIFF

In [12]:
def GetExtent(ds):
    """ Return list of corner coordinates from a gdal Dataset """
    xmin, xpixel, _, ymax, _, ypixel = ds.GetGeoTransform()
    width, height = ds.RasterXSize, ds.RasterYSize
    xmax = xmin + width * xpixel
    ymin = ymax + height * ypixel
    return round(xmin,0), round(xmax,0), round(ymin,0), round(ymax, 0)
# https://gis.stackexchange.com/questions/57834/how-to-get-raster-corner-coordinates-using-python-gdal-bindings

In [13]:
def crop(dataset_dir, final_dir, file_name):
    file_path = os.path.join(dataset_dir, file_name)
    
    name, exte = file_name.split('.')
    raster = gdal.Open(file_path)
    # print(file_path)
    ext = GetExtent(raster)
    Xdim, Ydim = raster.RasterXSize, raster.RasterYSize
    for i in range(Xdim//1024+1):
        for j in range(Ydim//1024+1):
            final_name = name[:-2]+ '_'+str(i)+str(j)+'.'+exte
            final_path = os.path.join(final_dir, final_name)
            
            if i==Xdim//1024 or j==Ydim//1024:
                if i==Xdim//1024 and j!=Ydim//1024:
                    nxmin = ext[1] - 1024 * 0.6
                    nxmax = ext[1]
                    nymin = ext[3] - 1024 * 0.6 * (j+1)
                    nymax = ext[3] - 1024 * 0.6 * j
                    #pxlim = [Xdim - 1024, Xdim, 0, 1024]
                    
                elif i!=Xdim//1024 and j==Ydim//1024:
                    nxmin = ext[0] + 1024 * 0.6 * i
                    nxmax = ext[0] + 1024 * 0.6 * (i+1)
                    nymin = ext[2]
                    nymax = ext[2] + 1024 * 0.6
                    #pxlim = [0, 1024, Ydim - 1024, Ydim]
                    
                elif i==Xdim//1024 and j==Ydim//1024:
                    nxmin = ext[1] - 1024 * 0.6
                    nxmax = ext[1]
                    nymin = ext[2]
                    nymax = ext[2] + 1024 * 0.6
                    #pxlim = [Xdim - 1024, Xdim, Ydim - 1024, Ydim]
                
            else:
                nxmin = ext[0] + 1024 * 0.6 * i
                nxmax = ext[0] + 1024 * 0.6 * (i+1)
                nymin = ext[3] - 1024 * 0.6 * (j+1)
                nymax = ext[3] - 1024 * 0.6 * j
                #pxlim = [1024*i, 1024*(i+1), 1024*j, 1024*(j+1)]
                
            !gdalwarp -overwrite -te $nxmin $nymin $nxmax $nymax $file_path $final_path
            
            #json_path = os.path.join(dataset_dir, name + '.json')
            #final_json_path = os.path.join(final_dir, name[:-2]+ '_'+str(i)+str(j)+'.json')
            #CropAnnot(json_path, final_json_path, final_path, final_name, pxlim)

In [3]:
tmp_path = os.path.join(dataset_dir, 'tmp')
print(tmp_path)

C:\Users\VArri\Documents\Rooftop\dataset\dataset\dataset\tmp


In [4]:
print(dataset_dir)

C:\Users\VArri\Documents\Rooftop\dataset\dataset\dataset


In [5]:
preprocess_path = os.path.join(dataset_dir, 'preprocess')
if not os.path.isdir(preprocess_path):
    os.mkdir(preprocess_path)


In [ ]:
lst = os.listdir(tmp_path)
for geo in lst: # 1 GeoTIFF preprocessed every 5 elements
    if geo.endswith('.tif'):
        with open(os.path.join(dataset_dir,'del.txt')) as file:
            lines = file.readlines()
            lines = [line.rstrip() for line in lines]
            if not any([geo.startswith(sub) for sub in lines]): 
                # si le nom du fichier ne commence par aucun des attributs de del.txt, alors on le préprocesse
                crop(tmp_path, preprocess_path, geo)

In [ ]:
import random
filepath = os.path.join(tmp_path, 'austin16_s.tif')
!gdalinfo $filepath

In [8]:
# Handling files in colab folder

colab_path = os.path.join(dataset_dir, 'colab2')
if not os.path.isdir(colab_path):
    os.mkdir(colab_path)
    
from shutil import copyfile
lst = os.listdir(preprocess_path)
for geo in lst:
    if geo.endswith('.tif'):
        name, exte = geo.split('.')
        if os.path.isfile(os.path.join(preprocess_path, name+'.json')):
            copyfile(os.path.join(preprocess_path,geo), os.path.join(colab_path,geo))
            copyfile(os.path.join(preprocess_path,name+'.json'), os.path.join(colab_path,name+'.json'))
